## Looking at the azimuth wrapping as a cause of oscillation fails

Craig Lage - 10-Apr-23

In [ ]:
import sys, time, os, asyncio
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler

In [ ]:
fig, ax1 = plt.subplots(1,1,figsize = (8,8))
ax1.plot(times, azs)
ax1.set_ylabel("Azimuth(degrees)")
ax1.set_xlabel("Time(seconds)")
ax2 = ax1.twinx()
mount_times = []
mount_motions = []
for seqNum in df.index.values.tolist():
    if seqNum >= 81 and seqNum <= 153:
        if seqNum in [89, 95, 108, 109, 127, 140]:
            continue
        time = Time(f"2023-04-13T{df.loc[seqNum]['TAI']}", format='isot', scale='tai')
        mount_times.append(time.utc.unix_tai - t0)
        mount_motions.append(df.loc[seqNum]['Mount motion image degradation'])
ax2.scatter(mount_times, mount_motions, marker='x', color='g')
ax2.set_ylabel("Mount jitter (arcseconds)")
plt.savefig(f"/home/craiglagegit/DATA/auxtel_azimuth_oscillations_13apr23/Both_Direction_Test_13Apr23.png")


In [ ]:
time = Time(f"2023-04-13T{df.loc[seqNum]['TAI']}", format='isot', scale='tai')

In [ ]:
time.utc.unix_tai

In [ ]:
az['azimuthCalculatedAngle'].plot()
plotStart = Time("2023-04-13 16:57:00Z", scale='utc').isot
plotEnd = Time("2023-04-13 16:59:00Z", scale='utc').isot
plt.xlim(plotStart, plotEnd)

In [ ]:
ave_tracking_torques = []
ave_slew_torques = []
mount_motions = []

for seqNum in range(82, 154):
    
    if seqNum in [89, 95, 108, 109, 127, 140]:
        continue
    expId = 2023041300000 + seqNum
    mount_motion = df.loc[seqNum]['Mount motion image degradation']
    fig, ax = plt.subplots(1,1,figsize = (8,8))
    az_torque['azimuthMotor1Torque'].plot(ax=ax)
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    expStart = Time(mData['DATE-BEG'], scale='tai')
    ax.axvline(expStart.utc.isot, color="g", linestyle="--", label='Exp Start')
    expEnd = Time(mData['DATE-END'], scale='tai')
    ax.axvline(expEnd.utc.isot, color="r", linestyle="--", label='Exp End')
    mDataLast = butler.get('raw.metadata', detector=0, exposure=expId-1)
    expEndLast = Time(mDataLast['DATE-END'], scale='tai')
    slewStart = expEndLast  + TimeDelta(5.0, format='sec')
    slewEnd = expEndLast  + TimeDelta(10.0, format='sec')
    ax.axvline(slewStart.utc.isot, color="magenta", linestyle="--", label='Slew Start')
    ax.axvline(slewEnd.utc.isot, color="cyan", linestyle="--", label='Slew End')
    plotStart = expStart + TimeDelta(-30.0, format='sec')
    plotEnd = expEnd + TimeDelta(30.0, format='sec')
    tracking_torque =  await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", "azimuthMotor1Torque", \
                                                              expStart.utc, expEnd.utc)
    ave_tracking_torque = np.mean(tracking_torque['azimuthMotor1Torque'].values)
    slew_torque =  await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", "azimuthMotor1Torque", \
                                                              slewStart.utc, slewEnd.utc)
    ave_slew_torque = np.mean(slew_torque['azimuthMotor1Torque'].values)
    print(seqNum, ave_tracking_torque, ave_slew_torque, mount_motion)
    #az_torque['azimuthMotor2Torque'].plot(ax=ax)
    plt.xlim(plotStart.utc.isot, plotEnd.utc.isot)
    plt.legend()
    plt.savefig(f"/home/craiglagegit/DATA/auxtel_azimuth_oscillations_13apr23/Torques_{expId}.png")
    plt.close()
    
    
    
    ave_tracking_torques.append(ave_tracking_torque)
    ave_slew_torques.append(ave_slew_torque)
    mount_motions.append(mount_motion)
    
    

In [ ]:
print(len(ave_tracking_torques), len(ave_slew_torques), len(mount_motions))

In [ ]:
plt.scatter(np.array(ave_tracking_torques) / np.array(ave_slew_torques), mount_motions)

In [ ]:
plt.scatter(abs(np.array(ave_tracking_torques) / np.array(ave_slew_torques)), mount_motions)

In [ ]:
plt.scatter(np.array(ave_tracking_torques), mount_motions)

In [ ]:
plt.scatter(np.array(ave_slew_torques), mount_motions)

In [ ]:
colors = ['r' if x > 0.25 else 'g' for x in mount_motions]
plt.title("Mount motion failures vs torques - 13Apr23", fontsize=18)
plt.scatter(ave_tracking_torques, ave_slew_torques, c=colors, alpha=0.5, s=100)
plt.xlabel('Tracking Torque (amps)')
plt.xlim(-1.5,1.5)
plt.ylabel('Slewing Torque (amps)')
plt.ylim(-1.5,1.5)
plt.savefig(f"/home/craiglagegit/DATA/auxtel_azimuth_oscillations_13apr23/Fails_vs_Torques_13Apr23.png")

In [ ]:
colors = ['r' if x > 0.25 else 'g' for x in mount_motions]

In [ ]:
colors

In [ ]:
plt.clf()
fig, [ax1,ax2] = plt.subplots(2,1,figsize = (16,8))
plt.subplots_adjust(hspace=0.5)
ax1.set_title("Impact of unwrapping on mount motion", fontsize = 18)
ax1.plot(times, azs, color='green', label="Azimuth")
ax1.set_ylabel("Degrees")
ax1.set_xlabel("Time (seconds)")

ax2.set_title("Azimuth Motor 1 Torque", fontsize=18)
ax2.plot(times, torques)
ax2.plot([times[0], times[-1]], [0.0,0.0], ls='--', color='red')
ax2.set_ylabel("Torque(amps)")
ax2.set_xlabel("Time (seconds)")


#plt.savefig(f"/home/craiglagegit/DATA/stiction_04apr23/Unwrapping_10Apr23.png")


In [ ]:
# Times to start looking at encoder values
start = Time("2023-04-11 14:31:40Z", scale='utc')
end = Time("2023-04-11 14:42:00Z", scale='utc')
plotStart = (start + TimeDelta(0.0, format='sec')).to_datetime()
plotEnd = (end + TimeDelta(0.0, format='sec')).to_datetime()

In [ ]:
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", "azimuthCalculatedAngle", start, end)
azs = az['azimuthCalculatedAngle'].values
times = az['times'].values
t0 = times[0]
times -= t0
tmax = times[-1]

az_torque_1 =  await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", "azimuthMotor1Torque", start, end)
torques = az_torque_1['azimuthMotor1Torque'].values

In [ ]:
azs[-1]

In [ ]:
plt.clf()
fig, [ax1,ax2] = plt.subplots(2,1,figsize = (16,8))
plt.subplots_adjust(hspace=0.5)
ax1.set_title("Impact of unwrapping on mount motion", fontsize = 18)
ax1.plot(times, azs, color='green', label="Azimuth")
ax1.set_ylabel("Degrees")
ax1.set_xlabel("Time (seconds)")

ax2.set_title("Azimuth Motor 1 Torque", fontsize=18)
ax2.plot(times, torques)
ax2.plot([times[0], times[-1]], [0.0,0.0], ls='--', color='red')
ax2.set_ylabel("Torque(amps)")
ax2.set_xlabel("Time (seconds)")


#plt.savefig(f"/home/craiglagegit/DATA/stiction_04apr23/Unwrapping_10Apr23.png")


In [ ]:
plt.figure()
plt.title("Azimuth Torque vs Angle - 11Apr23", fontsize=18)
plt.plot(azs, torques)
#plt.plot([-160,160],[3.0,3.0], color='red', ls='--')
#plt.plot([-160,160],[-3.0,-3.0], color='red', ls='--')
plt.arrow(-140, 1.5, 50,0, width=0.1,head_length = 5.0, color='green')
plt.arrow(140, -1.5, -50,0, width=0.1,head_length = 5.0, color='green')
plt.ylim(-2.0,2.0)
plt.xlabel("Azimuth(degrees)")
plt.ylabel("Torque (amps)")
plt.savefig("/home/craiglagegit/DATA/Azimuth_Torque_vs_Angle_11Apr23.pdf")